In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# %%
import pandas as pd

# %%
import re
import os
import time

# %%
import matplotlib.pyplot as plt
import argparse
import logging

In [2]:
!pip install --q opinf

In [3]:
import opinf as op

In [4]:
import numpy as np
import scipy.linalg as la
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from random import SystemRandom

In [5]:
%matplotlib inline

In [6]:
plt.rc("axes.spines", right=True, top=True)
plt.rc("figure", dpi=300, 
       figsize=(9, 3)
      )
plt.rc("font", family="serif")
plt.rc("legend", edgecolor="none", frameon=True)
# plt.style.use("dark_background")

In [7]:
import edge_utils as edut
import node_1d_utils as nut

In [8]:
# %%
# SPECIFY ARGS
# Generative model for noisy data based on ODE
parser = argparse.ArgumentParser('PNODE')

# parser.add_argument('-n',  type=int, default=1, help="Size of the dataset")
# parser.add_argument('-n',  type=int, default=100, help="Size of the dataset")

parser.add_argument('--niters', type=int, 
                    default=50)
parser.add_argument('--lr',  type=float, default=1e-2, help="Starting learning rate.")
parser.add_argument('-bs', '--batch-size', type=int, default=4)
parser.add_argument('-bt', '--batch-time', type=int, default=5)
parser.add_argument('-nspb', '--nsims_per_batch', type=int, default=1)
parser.add_argument('-epch', '--nepochs', type=int, default=300)


parser.add_argument('--save', type=str, default='experiments/', 
                    help="Path for save checkpoints")
parser.add_argument('--load', type=str, 
                    #                     default=None, 
                    default=86364,
                    help="ID of the experiment to load for evaluation. If None, run a new experiment.")
parser.add_argument('-r', '--random-seed', type=int, default=1123, help="Random_seed")


parser.add_argument('--pnode', 
                    action='store_false', 
                    help="RUN parameterized neural ode")
parser.add_argument('--node-layers', 
                    type=int, 
                    default=3, 
                    help="number of layers in NODE")

parser.add_argument('-u', '--units', 
                    type=int, 
                    default=90, 
                    help="Number of units per layer in ODE func")

parser.add_argument('--nParamsToUse',
                    type=int,
                    default=9,
                    help="Number of CME params to use")

# parser.add_argument('--normalize', type=bool, default=True)

parser.add_argument('-ds',
                    type=int,
                    default=1,
                    help="Coarsening factor for position angles")

# args = parser.parse_args(args=())
args = parser.parse_args(args=())

# %%
vars(args)

{'niters': 50,
 'lr': 0.01,
 'batch_size': 4,
 'batch_time': 5,
 'nsims_per_batch': 1,
 'nepochs': 300,
 'save': 'experiments/',
 'load': 86364,
 'random_seed': 1123,
 'pnode': True,
 'node_layers': 3,
 'units': 90,
 'nParamsToUse': 9,
 'ds': 1}

In [9]:
theta_s_2161 = np.linspace(0, 360, 512)[160] + 1.2 * 180 - 360
theta_e_2161 = np.linspace(0, 360, 512)[320] + 1.2 * 180 - 360
print("Range of angles for CR2161: {} {}".format(theta_s_2161, theta_e_2161))

# %%
ed_2161, sd_2161 = edut.load_edge_data_blobfree(2161)
# crude downsample for now
ed_2161 = ed_2161[:, ::args.ds, :]

# %%
nTimes, nTheta_2161, nSims_2161 = ed_2161.shape
nTimes, nTheta_2161, nSims_2161

# %%
theta_grid = np.linspace(np.ceil(theta_s_2161), np.ceil(theta_e_2161), nTheta_2161 * args.ds)[::args.ds]
theta_grid[:5]

Range of angles for CR2161: -31.279843444227026 81.44031311154595


array([-31.        , -30.28930818, -29.57861635, -28.86792453,
       -28.1572327 ])

In [10]:
theta_grid.shape

(160,)

In [11]:
def getRValuesAllSims(edge_data_matrix, sample_freq=2):
    """
    Return r values for all sims at once so we don't lose time in training processing r values repeatedly
    """
    r_data_matrix = np.zeros(edge_data_matrix.shape)
    nsims = edge_data_matrix.shape[2]
    for i in range(nsims):
        r_vals, theta_vals = edut.getRValues(edge_data_matrix, simIdx=i, minStartIdx=0, sample_freq=sample_freq)
        r_data_matrix[:, :, i] = r_vals

    return r_data_matrix


# %%
rd_2161 = getRValuesAllSims(ed_2161, sample_freq=1)

# %%
# we are removing some data where the edge detection is not necessarily super reliable.
sims_to_remove = np.array([33, 39, 63, 73, 113, 128, 131, 142, 193, 218, 253, 264, 273, 312, 313, 324])

# %%
sd_modified = np.setdiff1d(sd_2161, sims_to_remove)

In [12]:
rd_2161.shape

(90, 160, 278)

In [13]:
from numpy.random import Generator, PCG64
# rng = Generator(PCG64())
rng = np.random.default_rng(seed=202310)

nTrain = int(np.floor(0.7 * len(sd_modified)))
nCalib = int(np.floor(0.15 * len(sd_modified)))
nTest = len(sd_modified) - nTrain - nCalib

print(nTrain, nCalib, nTest)

sd_train = np.sort(rng.choice(sd_modified, nTrain, replace=False))
sd_calib = np.sort(rng.choice(np.setdiff1d(sd_modified, sd_train), nCalib, replace=False))
sd_test = np.setdiff1d(sd_modified, np.sort(np.concatenate((sd_train, sd_calib), axis=0)))

183 39 40


In [14]:
orig_sd_train_idx = np.array([np.where(sd_2161 == i)[0][0] for i in sd_train])
orig_sd_test_idx = np.array([np.where(sd_2161 == i)[0][0] for i in sd_test])
orig_sd_calib_idx = np.array([np.where(sd_2161 == i)[0][0] for i in sd_calib])

In [15]:
y_train = rd_2161[:, :, orig_sd_train_idx]
y_test = rd_2161[:, :, orig_sd_test_idx]
y_calib = rd_2161[:, :, orig_sd_calib_idx]

In [16]:
tMinTrain = []
tMaxTrain = []

tMinTrainIdx=[]
tMaxTrainIdx=[]

dtTrain = []
yMinTrain = []
yMaxTrain = []

tMinTest = []
tMaxTest = []
dtTest = []

yMinTest = []
yMaxTest = []

tMinTestIdx = []
tMaxTestIdx = []

tMinCalib = []
tMaxCalib = []
dtCalib = []

for sidx in orig_sd_train_idx:
    
    r_sim = rd_2161[:, :, sidx]
    
    tMinIdx, tMin, tMaxIdx, tMax = edut.getTMinTMax(ed_2161, simIdx = sidx)

    r_sim_valid = r_sim[tMinIdx:(tMaxIdx + 1), :]
    
    tMinTrain.append(tMin)
    tMaxTrain.append(tMax)

    tMinTrainIdx.append(tMinIdx)
    tMaxTrainIdx.append(tMaxIdx)

    
    yMinTrain.append(r_sim_valid.min())
    yMaxTrain.append(r_sim_valid.max())
    
    tAllScaled = (np.arange(tMin, tMax + 2, step=2) - tMin) / (tMax - tMin)
    
    dtTrain.append(tAllScaled[1] - tAllScaled[0])
    
    
for sidx in orig_sd_test_idx:
    
    r_sim = rd_2161[:, :, sidx]
    
    tMinIdx, tMin, tMaxIdx, tMax = edut.getTMinTMax(ed_2161, simIdx = sidx)

    r_sim_valid = r_sim[tMinIdx:(tMaxIdx + 1), :]
    
    tMinTest.append(tMin)
    tMaxTest.append(tMax)
    
    tMinTestIdx.append(tMinIdx)
    tMaxTestIdx.append(tMaxIdx)
    
    yMinTest.append(r_sim_valid.min())
    yMaxTest.append(r_sim_valid.max())
        
    tAllScaled = (np.arange(tMin, tMax + 2, step=2) - tMin) / (tMax - tMin)
    
    dtTest.append(tAllScaled[1] - tAllScaled[0])
    
    
    
for sidx in orig_sd_calib_idx:    
    r_sim = rd_2161[:, :, sidx]
    
    tMinIdx, tMin, tMaxIdx, tMax = edut.getTMinTMax(ed_2161, simIdx = sidx)

    r_sim_valid = r_sim[tMinIdx:(tMaxIdx + 1), :]
    
    tMinCalib.append(tMin)
    tMaxCalib.append(tMax)
        
    tAllScaled = (np.arange(tMin, tMax + 2, step=2) - tMin) / (tMax - tMin)
    
    dtCalib.append(tAllScaled[1] - tAllScaled[0])

In [17]:
min(tMaxTrain)

124.0

In [18]:
max(tMinTrain)

86.0

In [19]:
yMinTest[9], yMinTest[20], yMinTest[39]

(3.7753038926141387, 3.7849222771356716, 3.782086075450167)

In [20]:
yMaxTest[9], yMaxTest[20], yMaxTest[39]

(11.952783574317628, 22.157402265402517, 17.18293718204489)

In [21]:
all_times_orig = np.linspace(2, 180, 90)
np.where(all_times_orig == min(tMaxTrain))[0][0]

61

In [22]:
np.where(all_times_orig == max(tMinTrain))[0][0]

42

In [23]:
# NOW NORMALIZE YTRAIN AND YTEST
yMinTrainAll = np.array(yMinTrain).min()
yMaxTrainAll = np.array(yMaxTrain).max()

yMinTrainAll, yMaxTrainAll

# %%
y_train_normalized = (y_train - yMinTrainAll) / (yMaxTrainAll - yMinTrainAll)
y_test_normalized = (y_test - yMinTrainAll) / (yMaxTrainAll - yMinTrainAll)

# %%
y_calib_normalized = (y_calib - yMinTrainAll) / (yMaxTrainAll - yMinTrainAll)

In [24]:
y_train_normalized.shape

(90, 160, 183)

In [25]:
yMinTrainAll

3.7698396083568695

In [26]:
yMaxTrainAll

22.359171674874087

In [27]:
# %%
cme_params_norm = pd.read_csv("./CMEParams2161_Scaled.csv", index_col=0)
cme_params_norm

# %%
cme_params_to_augment = cme_params_norm.to_numpy()
cme_params_to_augment.shape

(278, 9)

In [28]:
cme_params_norm

,Radius,BStrength,ApexHeight,OrientationCme,iHelicity,realization,FactorB0,PoyntingFluxPerBSi,LperpTimesSqrtBSi
0,0.5675,0.420040,0.568916,0.5525,1.0,0.272727,0.120833,0.098542,0.436506
1,0.4525,0.057350,0.348039,0.1025,1.0,0.272727,0.120833,0.098542,0.436506
2,0.8525,0.014777,0.541725,0.8775,1.0,0.272727,0.120833,0.098542,0.436506
3,0.1225,0.323231,0.214567,0.5075,0.0,0.272727,0.120833,0.098542,0.436506
4,0.7825,0.085414,0.640815,0.3325,1.0,0.272727,0.120833,0.098542,0.436506
...,...,...,...,...,...,...,...,...,...
294,0.8750,0.252834,0.622193,0.1750,0.0,0.363636,0.352593,0.005643,0.937194
295,0.1250,0.655648,0.235434,0.4750,0.0,0.363636,0.352593,0.005643,0.937194
296,0.3250,0.518360,0.245710,0.7750,1.0,0.363636,0.352593,0.005643,0.937194
297,0.9250,0.289994,0.829820,0.6750,0.0,0.363636,0.352593,0.005643,0.937194


In [29]:
cme_params_to_augment

array([[0.5675    , 0.42003969, 0.56891578, ..., 0.12083333, 0.09854212,
        0.43650556],
       [0.4525    , 0.05735036, 0.34803867, ..., 0.12083333, 0.09854212,
        0.43650556],
       [0.8525    , 0.01477748, 0.54172511, ..., 0.12083333, 0.09854212,
        0.43650556],
       ...,
       [0.325     , 0.51835972, 0.24570975, ..., 0.35259259, 0.005643  ,
        0.9371937 ],
       [0.925     , 0.28999395, 0.82981992, ..., 0.35259259, 0.005643  ,
        0.9371937 ],
       [0.625     , 0.40661486, 0.39412076, ..., 0.35259259, 0.005643  ,
        0.9371937 ]])

In [30]:
input_dim = rd_2161.shape[1]
param_dim = 9
input_dim, param_dim

(160, 9)

In [31]:
# augmented_r = np.zeros((rd_2161.shape[0], input_dim + param_dim, rd_2161.shape[2]))
# augmented_r[:, :input_dim, orig_sd_train_idx] = y_train_normalized
# augmented_r[:, :input_dim, orig_sd_test_idx] = y_test_normalized
# augmented_r[:, :input_dim, orig_sd_calib_idx] = y_calib_normalized
# for iii in range(rd_2161.shape[2]):
#     augmented_r[:, (input_dim):, iii] = cme_params_to_augment[iii, :]

In [32]:
# aug_y_train.shape

In [33]:
sd_train

array([ 31,  32,  34,  37,  38,  40,  41,  44,  45,  46,  49,  50,  51,
        53,  54,  56,  57,  58,  59,  61,  62,  67,  69,  70,  71,  72,
        75,  76,  77,  78,  79,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  97,  98, 101, 103, 104, 105, 107,
       110, 111, 112, 115, 118, 120, 123, 126, 127, 132, 134, 135, 136,
       138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 150, 151, 152,
       153, 154, 155, 156, 157, 159, 160, 161, 163, 164, 166, 171, 172,
       173, 175, 176, 177, 178, 180, 183, 184, 185, 190, 191, 192, 198,
       199, 201, 202, 203, 204, 205, 206, 213, 214, 215, 216, 217, 219,
       220, 221, 222, 223, 224, 226, 228, 229, 230, 231, 232, 233, 234,
       235, 236, 238, 239, 240, 241, 242, 243, 246, 247, 249, 251, 252,
       254, 255, 259, 260, 261, 265, 269, 271, 274, 275, 276, 278, 280,
       281, 282, 283, 284, 286, 288, 290, 293, 294, 295, 296, 299, 300,
       303, 305, 307, 314, 316, 317, 318, 319, 321, 322, 326, 32

In [34]:
# aug_y_train = augmented_r[:, :, orig_sd_train_idx]
# aug_y_test = augmented_r[:, :, orig_sd_test_idx]
# aug_y_calib = augmented_r[:, :, orig_sd_calib_idx]

# aug_y_train_rs = np.transpose(aug_y_train, (1, 0, 2))
# aug_y_train_rs.shape

# aug_y_train_transposed = np.transpose(aug_y_train_rs, (0, 2, 1))
# aug_y_train_unfolded = aug_y_train_transposed.reshape((169, 90 * 183))
# # aug_y_train_unfolded[160:, 270:275]

In [35]:
# aug_y_test_rs = np.transpose(aug_y_test, (1, 0, 2))
# aug_y_test_rs.shape

In [36]:
# aug_y_train_rs.shape

In [37]:
# aug_y_train_unfolded.shape

In [38]:
# Prior to basis fitting, set TMin, TMax correctly for each sim and assemble them into different matrices.
# These are concatenated to give the giant matrix that we compress.

In [39]:
# y_unfolded_train = []
# aug_y_unfolded_train = []

# ytn_rs = np.transpose(y_train_normalized, (1, 0, 2))
# for idx, i in enumerate(orig_sd_train_idx):
#     tMinIdx, tMin, tMaxIdx, tMax = edut.getTMinTMax(ed_2161, simIdx = i)
#     y_unfolded_train.append(ytn_rs[:, tMinIdx:(tMaxIdx + 1), idx])
#     aug_y_unfolded_train.append(aug_y_train_rs[:, tMinIdx:(tMaxIdx + 1), idx])

# y_unfolded_train_full = np.concatenate(y_unfolded_train, axis=1)
# aug_y_unfolded_train_full = np.concatenate(aug_y_unfolded_train, axis=1)

In [40]:
# y_unfolded_train_full.shape, aug_y_unfolded_train_full.shape

In [41]:
# basis = op.basis.PODBasis().fit(aug_y_unfolded_train_full[:160, :], residual_energy=1e-8)
# basis = op.basis.PODBasis().fit(y_unfolded_train_full, residual_energy=1e-8)

In [42]:
# print(basis)

In [43]:
sd_2161[orig_sd_test_idx]

array([ 35,  43,  66,  68,  81,  96,  99, 106, 108, 114, 119, 122, 124,
       133, 158, 174, 181, 186, 188, 194, 200, 208, 212, 237, 244, 248,
       256, 257, 258, 266, 267, 268, 277, 279, 287, 292, 302, 306, 311,
       323])

### Ignore the above

Pick 3 test simulations and fit ROMs to all of them

In [44]:
augmented_r = np.zeros((rd_2161.shape[0], input_dim + param_dim, rd_2161.shape[2]))
augmented_r[:, :input_dim, orig_sd_train_idx] = y_train
augmented_r[:, :input_dim, orig_sd_test_idx] = y_test
augmented_r[:, :input_dim, orig_sd_calib_idx] = y_calib
for iii in range(rd_2161.shape[2]):
    augmented_r[:, (input_dim):, iii] = cme_params_to_augment[iii, :]

In [45]:
aug_y_train = augmented_r[:, :, orig_sd_train_idx]
aug_y_test = augmented_r[:, :, orig_sd_test_idx]
aug_y_calib = augmented_r[:, :, orig_sd_calib_idx]

In [46]:
orig_sd_test_idx[9], orig_sd_test_idx[20], orig_sd_test_idx[39]

(81, 156, 271)

In [47]:
sd_2161[[81, 156, 271]]

array([114, 200, 323])

In [48]:
tMinSelected = np.array([tMinTestIdx[9], tMinTestIdx[20], tMinTestIdx[39]])

In [49]:
tMaxSelected = np.array([tMaxTestIdx[9], tMaxTestIdx[20], tMaxTestIdx[39]])

In [50]:
trainExtents = np.floor(0.6 * (tMaxSelected - tMinSelected)).astype(int)

In [51]:
trainExtents

array([38, 30, 44])

In [52]:
tMinSelected

array([25, 18, 15])

In [53]:
tTrainStop = tMinSelected + trainExtents
tTrainStop

array([63, 48, 59])

In [54]:
tMaxSelected

array([89, 69, 89])

In [55]:
aug_y_test.shape

(90, 169, 40)

In [56]:
yMinsAll = [yMinTest[9], yMinTest[20], yMinTest[39]]
yMaxsAll = [yMaxTest[9], yMaxTest[20], yMaxTest[39]]

In [57]:
aug_y_ts1 = (aug_y_test[tMinSelected[0]:(tMaxSelected[0] + 1), :160, 9].T - yMinTest[9])/(yMaxTest[9] - yMinTest[9])
aug_y_ts2 = (aug_y_test[tMinSelected[1]:(tMaxSelected[1] + 1), :160, 20].T - yMinTest[20])/(yMaxTest[20] - yMinTest[20])
aug_y_ts3 = (aug_y_test[tMinSelected[2]:(tMaxSelected[2] + 1), :160, 39].T - yMinTest[39])/(yMaxTest[39] - yMinTest[39])
aug_y_ts1.shape, aug_y_ts2.shape, aug_y_ts3.shape

((160, 65), (160, 52), (160, 75))

In [58]:
aug_y_ts1_train = aug_y_ts1[:, :(trainExtents[0])]
aug_y_ts2_train = aug_y_ts2[:, :(trainExtents[1])] 
aug_y_ts3_train = aug_y_ts3[:, :(trainExtents[2])] 

aug_y_ts1_train.shape, aug_y_ts2_train.shape, aug_y_ts3_train.shape

((160, 38), (160, 30), (160, 44))

In [59]:
aug_y_ts1_test = aug_y_ts1[:, (trainExtents[0]):(tMaxSelected[0] + 1)]
aug_y_ts2_test = aug_y_ts2[:, (trainExtents[1]):(tMaxSelected[1] + 1)]
aug_y_ts3_test = aug_y_ts3[:, (trainExtents[2]):(tMaxSelected[2] + 1)]

aug_y_ts1_test.shape, aug_y_ts2_test.shape, aug_y_ts3_test.shape

((160, 27), (160, 22), (160, 31))

In [60]:
basis1 = op.basis.PODBasis().fit(aug_y_ts1_train, residual_energy=1e-8)
basis2 = op.basis.PODBasis().fit(aug_y_ts2_train, residual_energy=1e-8)
basis3 = op.basis.PODBasis().fit(aug_y_ts3_train, residual_energy=1e-8)

In [61]:
basis1, basis2, basis3

(<PODBasis object at 0x7f204e80c4f0>
 PODBasis
 Full-order dimension    n = 160
 Reduced-order dimension r = 28,
 PODBasis
 Full-order dimension    n = 160
 Reduced-order dimension r = 14,
 PODBasis
 Full-order dimension    n = 160
 Reduced-order dimension r = 18)

In [62]:
# Instantiate the model.
rom1 = op.models.ContinuousModel(operators=[
    op.operators.LinearOperator(),
    op.operators.QuadraticOperator(),
])

rom2 = op.models.ContinuousModel(operators=[
    op.operators.LinearOperator(),
    op.operators.QuadraticOperator(),
])

rom3 = op.models.ContinuousModel(operators=[
    op.operators.LinearOperator(),
    op.operators.QuadraticOperator(),
])

In [63]:
t1 = np.linspace(0, 1, 65)
dt1 = t1[1] - t1[0]

t2 = np.linspace(0, 1, 52)
dt2 = t2[1] - t2[0]

t3 = np.linspace(0, 1, 75)
dt3 = t3[1] - t3[0]

In [64]:
qc1 = basis1.compress(aug_y_ts1_train)
qTrain1 = qc1[:, 1:]
qDotTrain1 = (qc1[:, 1:] - qc1[:, :-1])/dt1

In [65]:
rom1.fit(qTrain1, qDotTrain1, solver=op.lstsq.L2Solver(regularizer=5))

/home/ajivani/miniconda3/lib/python3.10/site-packages/opinf/lstsq/_tikhonov.py:119: LinAlgWarning: non-regularized least-squares system is underdetermined!
  _BaseTikhonovSolver.fit(self, A, B)


Model structure: dq / dt = Aq(t) + H[q(t) ⊗ q(t)]
State dimension r = 28

In [66]:
qc2 = basis2.compress(aug_y_ts2_train)
qTrain2 = qc2[:, 1:]
qDotTrain2 = (qc2[:, 1:] - qc2[:, :-1])/dt2

rom2.fit(qTrain2, qDotTrain2, solver=op.lstsq.L2Solver(regularizer=0.8))

Model structure: dq / dt = Aq(t) + H[q(t) ⊗ q(t)]
State dimension r = 14

In [67]:
qc3 = basis3.compress(aug_y_ts3_train)
qTrain3 = qc3[:, 1:]
qDotTrain3 = (qc3[:, 1:] - qc3[:, :-1])/dt3

rom3.fit(qTrain3, qDotTrain3, solver=op.lstsq.L2Solver(regularizer=5))

Model structure: dq / dt = Aq(t) + H[q(t) ⊗ q(t)]
State dimension r = 18

In [68]:
yc_pred_1 = rom1.predict(qc1[:, 0], t1)
yc_pred_1_dc = basis1.decompress(yc_pred_1)

In [69]:
yc_pred_1_dc.shape

(160, 65)

In [70]:
yc_pred_2 = rom2.predict(qc2[:, 0], t2)
yc_pred_2_dc = basis2.decompress(yc_pred_2)

yc_pred_2_dc.shape

(160, 52)

In [71]:
yc_pred_3 = rom3.predict(qc3[:, 0], t3)
yc_pred_3_dc = basis3.decompress(yc_pred_3)

yc_pred_3_dc.shape

(160, 75)

In [72]:
# all_times[:(tMaxSelected[2] - tMinSelected[2] + 1)]

In [73]:
np.savez("opinf_siam_predictions.npz", pred1=yc_pred_1_dc, pred2 = yc_pred_2_dc, pred3=yc_pred_3_dc,
        truth1=aug_y_ts1, truth2=aug_y_ts2, truth3=aug_y_ts3,
        yMinsAll = yMinsAll, yMaxsAll=yMaxsAll
        )

Ignore everything below this

In [ ]:
# pass through basis, 

In [ ]:
# svdvals = basis.svdvals

In [ ]:
# plt.plot(svdvals / svdvals[0])

In [ ]:
compressed_train_states = []
compressed_lhs = []
for i in range(len(orig_sd_train_idx)):
    # create s rxk arrays, where s is number of param vectors, r is reduced dimension, k is number of train timesteps.
    
    qcompressed_i = basis.compress(aug_y_unfolded_train[i][:160, :])
    
    qtrain_i = qcompressed_i[:, 1:]
    zero_params = np.zeros((9, qtrain_i.shape[1]))
    
    compressed_train_states.append(qtrain_i)
    qdot_train_i = (qcompressed_i[:, 1:] - qcompressed_i[:, :-1])/dt
    
#     compressed_train_states.append(np.concatenate((qtrain_i, aug_y_unfolded_train[i][160:, 1:]), axis=0))
#     qdot_train_i = np.concatenate(((qcompressed_i[:, 1:] - qcompressed_i[:, :-1])/dt, zero_params), axis=0)
    compressed_lhs.append(qdot_train_i)
    
compressed_train_all = np.concatenate(compressed_train_states, axis=1)
compressed_ddts_all = np.concatenate(compressed_lhs, axis=1)

In [ ]:
# all_cme_params_train = []
# for i in range(len(orig_sd_train_idx)):
#     all_cme_params_train.append(cme_params_to_augment[orig_sd_train_idx[i], [0, 4]])

In [ ]:
# all_cme_params_train

In [ ]:
# all_cme_params_test

In [ ]:
# all_cme_params_train[0].shape

In [ ]:
all_times = np.linspace(0, 1, 90)

In [ ]:
dt = all_times[1] - all_times[0]

In [ ]:
compressed_train_states = []
compressed_lhs = []
for i in range(len(orig_sd_train_idx)):
    # create s rxk arrays, where s is number of param vectors, r is reduced dimension, k is number of train timesteps.
    
    qcompressed_i = basis.compress(aug_y_unfolded_train[i][:160, :])
    
    qtrain_i = qcompressed_i[:, 1:]
    zero_params = np.zeros((9, qtrain_i.shape[1]))
    
    compressed_train_states.append(qtrain_i)
    qdot_train_i = (qcompressed_i[:, 1:] - qcompressed_i[:, :-1])/dt
    
#     compressed_train_states.append(np.concatenate((qtrain_i, aug_y_unfolded_train[i][160:, 1:]), axis=0))
#     qdot_train_i = np.concatenate(((qcompressed_i[:, 1:] - qcompressed_i[:, :-1])/dt, zero_params), axis=0)
    compressed_lhs.append(qdot_train_i)
    
compressed_train_all = np.concatenate(compressed_train_states, axis=1)
compressed_ddts_all = np.concatenate(compressed_lhs, axis=1)

In [ ]:
compressed_train_states[0].shape

In [ ]:
compressed_lhs[0].shape, compressed_train_states[0].shape

In [ ]:
basis = op.basis.PODBasis().fit(aug_y_unfolded_train[0][:160, :40], residual_energy=1e-8)

In [ ]:
compressed_lhs[1].shape, compressed_train_states[1].shape

In [ ]:
# rom.fit(all_cme_params_train, compressed_train_states, compressed_lhs)
# rom.fit(compressed_train_all, compressed_ddts_all)

In [ ]:
rom.A_.entries.shape

In [ ]:
rom.H_.entries.shape

In [ ]:
# y_unfolded_test = []
# ytest_rs = np.transpose(y_test_normalized, (1, 0, 2))
aug_y_test_full = []
for idx, i in enumerate(orig_sd_test_idx):
    tMinIdx, tMin, tMaxIdx, tMax = edut.getTMinTMax(ed_2161, simIdx = i)
    aug_y_test_full.append(aug_y_test_rs[:, tMinIdx:(tMaxIdx + 1), idx])

aug_y_unfolded_test_full = np.concatenate(aug_y_test_full, axis=1)

In [ ]:
# construct Initial Condition, append parameters, and send it through to predictions.
all_test_predictions = []

for idx, i in enumerate(orig_sd_test_idx):
    tMinIdx, tMin, tMaxIdx, tMax = edut.getTMinTMax(ed_2161, simIdx = i)
    print(tMinIdx, tMaxIdx)
    y0_test_i = aug_y_test_full[idx][:160, 0]
    y0_test_ic = basis.compress(y0_test_i)
    y0_test_aug = np.concatenate((y0_test_ic, aug_y_test_full[idx][160:, 0]), axis=0)
    yt_test_i = rom.predict(y0_test_aug, all_times[:(tMaxIdx - tMinIdx + 1)])
    yt_test_dc = basis.decompress(yt_test_i[:38, :])
    all_test_predictions.append(yt_test_dc)

In [ ]:
all_test_predictions[0]

In [ ]:
# all_cme_params_test = []
# for i in range(len(orig_sd_test_idx)):
#     all_cme_params_test.append(cme_params_to_augment[orig_sd_test_idx[i], [0, 4]])
    
#     rom.predict(all_cme_params_test[idx], y0_test_ic, all_times[:(tMaxIdx - tMinIdx + 1)])

In [ ]:
y_unfolded_test[1].shape

In [ ]:
all_cme_params_train[0]

In [ ]:
all_cme_params_test[0]

In [ ]:
# tMinIdx, tMin, tMaxIdx, tMax = edut.getTMinTMax(ed_2161, simIdx = orig_sd_test_idx[0])
# tMinIdx, tMaxIdx

In [ ]:
# all_times = np.linspace(2, 180, 90)

In [ ]:
# y0_test_i = y_unfolded_test[0][:, 0]
# y0_test_ic = basis.compress(y0_test_i)
# y0_test_ic.shape

In [ ]:
# type(all_cme_params_test[0])

In [ ]:
# plt.imshow(rom.A_.entries[25])
# plt.colorbar()

In [ ]:
# rom.evaluate(all_cme_params_train[0])

In [ ]:
# for idx, i in enumerate(orig_sd_test_idx):
#     model_at_parameter = rom.evaluate(all_cme_params_test[idx].reshape((6, )))

In [ ]:
all_test_predictions = []

for idx, i in enumerate(orig_sd_test_idx):
    tMinIdx, tMin, tMaxIdx, tMax = edut.getTMinTMax(ed_2161, simIdx = i)
    y0_test_i = y_unfolded_test[idx][:, 0]
    y0_test_ic = basis.compress(y0_test_i)
    yt_test_i = rom.predict(all_cme_params_test[idx], y0_test_ic, all_times[:(tMaxIdx - tMinIdx + 1)])
    yt_test_dc = basis.decompress(yt_test_i)
    all_test_predictions.append(yt_test_dc)
# q0_r = basis_all.compress(q0_new)
# Q_ROM_new = basis.decompress(implicit_euler(t, q0_, rom.A_.entries, rom.B_.entries, U_all))
# qt_new = rom_interp.predict(np.array(param_new), q0_r, t, input_func=np.ones_like(t))

In [ ]:
# # Create a 4x3x2 tensor as an example
# tensor = np.arange(4 * 3 * 2).reshape((4, 3, 2))

# # First, we transpose the tensor to bring the last axis to the second position
# transposed = np.transpose(tensor, (0, 2, 1))

# # Then, we reshape the tensor to a 4x6 matrix
# matrix = transposed.reshape((4, 6))

# print(tensor[:, :, 0])
# print(tensor[:, :, 1])

# print(matrix)